# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Big Data Real-Time Analytics com Python e Spark</font>

## <font color='blue'>Lab 5</font>

## <font color='blue'>Machine Learning com PySpark</font>

Leia os manuais em pdf no Capítulo 14 do curso com o material complementar.

![title](imagens/Lab5.png)

## <font color='blue'>Classificação Multiclasse</font>

Usaremos a Classificação Multiclasse com Árvores de Decisão para construir um modelo capaz de prever o resultado de uma partida de futebol com 3 resultados possíveis: vitória, derrota ou empate.

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.7


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
#!pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
#!pip install -q -U watermark

In [3]:
# Importa o findspark e inicializa
import findspark
findspark.init()

In [4]:
# Imports 
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

findspark: 2.0.1
pyspark  : 3.3.0



## Carregando os Dados

In [6]:
# Criando o Spark Context
sc = SparkContext(appName = "Lab5")

22/08/24 17:52:46 WARN Utils: Your hostname, falcon.local resolves to a loopback address: 127.0.0.1; using 10.0.0.87 instead (on interface en0)
22/08/24 17:52:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/24 17:52:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/24 17:52:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
sc.setLogLevel("ERROR")

In [8]:
# Spark Session - usada quando se trabalha com Dataframes no Spark
spSession = SparkSession.builder.master("local").getOrCreate()

In [9]:
# Carregando os dados e gerando um RDD
dados_time_futebol = sc.textFile("dados/dataset2.csv")

In [10]:
# Colocando o RDD em cache. Esse processo otimiza a performance.
dados_time_futebol.cache()

dados/dataset2.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [11]:
dados_time_futebol.count()

151

In [12]:
dados_time_futebol.take(5)

['media_faltas_sofridas,media_faltas_recebidas,media_cartoes_recebidos,media_chutes_a_gol,resultado',
 '4.8,3,1.4,0.3,vitoria',
 '5.1,3.8,1.6,0.2,vitoria',
 '4.6,3.2,1.4,0.2,vitoria',
 '5.3,3.7,1.5,0.2,vitoria']

In [13]:
# Removendo a primeira linha do arquivo (cabeçalho)
dados_time_futebol_2 = dados_time_futebol.filter(lambda x: "media_faltas_sofridas" not in x)
dados_time_futebol_2.count()

150

## Limpeza e Transformação dos Dados

In [14]:
# Separando as colunas
dados_time_futebol_3 = dados_time_futebol_2.map(lambda l: l.split(","))

In [15]:
# Mapeando as colunas
dados_time_futebol_4 = dados_time_futebol_3.map(lambda p: Row(media_faltas_sofridas = float(p[0]), 
                                                              media_faltas_recebidas = float(p[1]), 
                                                              media_cartoes_recebidos = float(p[2]), 
                                                              media_chutes_a_gol = float(p[3]), 
                                                              resultado = p[4] ))

In [16]:
# Converte o RDD para DataFrame do Spark
df_time = spSession.createDataFrame(dados_time_futebol_4)
df_time.cache()

DataFrame[media_faltas_sofridas: double, media_faltas_recebidas: double, media_cartoes_recebidos: double, media_chutes_a_gol: double, resultado: string]

In [17]:
df_time.take(5)

[Row(media_faltas_sofridas=4.8, media_faltas_recebidas=3.0, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.3, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.8, media_cartoes_recebidos=1.6, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=4.6, media_faltas_recebidas=3.2, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.3, media_faltas_recebidas=3.7, media_cartoes_recebidos=1.5, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.5, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria')]

In [18]:
# Criando um índice numérico para a coluna de label target
stringIndexer = StringIndexer(inputCol = "resultado", outputCol = "idx_resultado")

In [19]:
# Treinando o string indexer
si_model = stringIndexer.fit(df_time)

In [20]:
# Aplicando o string indexer
df_time_final = si_model.transform(df_time)

In [21]:
df_time_final.select("resultado", "idx_resultado").distinct().collect()

[Row(resultado='derrota', idx_resultado=0.0),
 Row(resultado='vitoria', idx_resultado=2.0),
 Row(resultado='empate', idx_resultado=1.0)]

## Análise Exploratória de Dados

In [22]:
# Estatística descritiva
df_time_final.describe().show()

+-------+---------------------+----------------------+-----------------------+------------------+---------+------------------+
|summary|media_faltas_sofridas|media_faltas_recebidas|media_cartoes_recebidos|media_chutes_a_gol|resultado|     idx_resultado|
+-------+---------------------+----------------------+-----------------------+------------------+---------+------------------+
|  count|                  150|                   150|                    150|               150|      150|               150|
|   mean|    5.843333333333332|    3.0573333333333337|      3.758000000000001|1.1993333333333331|     null|               1.0|
| stddev|   0.8280661279778625|   0.43586628493669793|     1.7652982332594667|0.7622376689603465|     null|0.8192319205190404|
|    min|                  4.3|                   2.0|                    1.0|               0.1|  derrota|               0.0|
|    max|                  7.9|                   4.4|                    6.9|               2.5|  vitoria|    

In [23]:
# Correlação entre as variáveis
for i in df_time_final.columns:
    if not(isinstance(df_time_final.select(i).take(1)[0][0], str)) :
        print("Correlação da variável idx_resultado com:", i, df_time_final.stat.corr('idx_resultado', i))

Correlação da variável idx_resultado com: media_faltas_sofridas -0.460039156500237
Correlação da variável idx_resultado com: media_faltas_recebidas 0.6183715308237437
Correlação da variável idx_resultado com: media_cartoes_recebidos -0.6492418307641741
Correlação da variável idx_resultado com: media_chutes_a_gol -0.5803770334306263
Correlação da variável idx_resultado com: idx_resultado 1.0


## Pré-Processamento dos Dados

In [24]:
# Criando um LabeledPoint (target, Vector[features])
# Remove colunas não relevantes para o modelo ou com baixa correlação
def transformaVar(row) :
    obj = (row["resultado"], row["idx_resultado"], Vectors.dense([row["media_faltas_sofridas"], 
                                                                  row["media_faltas_recebidas"],
                                                                  row["media_cartoes_recebidos"], 
                                                                  row["media_chutes_a_gol"]]))
    return obj

In [25]:
# Aplica a função
df_time_final_RDD = df_time_final.rdd.map(transformaVar)

In [26]:
df_time_final_RDD.take(5)

[('vitoria', 2.0, DenseVector([4.8, 3.0, 1.4, 0.3])),
 ('vitoria', 2.0, DenseVector([5.1, 3.8, 1.6, 0.2])),
 ('vitoria', 2.0, DenseVector([4.6, 3.2, 1.4, 0.2])),
 ('vitoria', 2.0, DenseVector([5.3, 3.7, 1.5, 0.2])),
 ('vitoria', 2.0, DenseVector([5.1, 3.5, 1.4, 0.2]))]

In [27]:
# Converte o RDD para DataFrame
df_spark = spSession.createDataFrame(df_time_final_RDD,["resultado", "label", "features"])

In [28]:
df_spark.cache()

DataFrame[resultado: string, label: double, features: vector]

In [29]:
df_spark.select("resultado", "label", "features").show(10)

+---------+-----+-----------------+
|resultado|label|         features|
+---------+-----+-----------------+
|  vitoria|  2.0|[4.8,3.0,1.4,0.3]|
|  vitoria|  2.0|[5.1,3.8,1.6,0.2]|
|  vitoria|  2.0|[4.6,3.2,1.4,0.2]|
|  vitoria|  2.0|[5.3,3.7,1.5,0.2]|
|  vitoria|  2.0|[5.1,3.5,1.4,0.2]|
|  vitoria|  2.0|[4.9,3.0,1.4,0.2]|
|  vitoria|  2.0|[4.7,3.2,1.3,0.2]|
|  vitoria|  2.0|[4.6,3.1,1.5,0.2]|
|  vitoria|  2.0|[5.0,3.6,1.4,0.2]|
|  vitoria|  2.0|[5.4,3.9,1.7,0.4]|
+---------+-----+-----------------+
only showing top 10 rows



In [30]:
# Dados de Treino e de Teste
(dados_treino, dados_teste) = df_spark.randomSplit([0.7, 0.3])

In [31]:
dados_treino.count()

116

In [32]:
dados_teste.count()

34

## Machine Learning

In [33]:
# Cria o objeto
dtClassifer = DecisionTreeClassifier(maxDepth = 2, labelCol = "label", featuresCol = "features")

In [34]:
# Treina o objeto com dados para criar o modelo
modelo = dtClassifer.fit(dados_treino)

In [35]:
# Hiperparâmetro definido por padrão
modelo.numNodes

5

In [36]:
# Hiperparâmetro definido por nós
modelo.depth

2

In [37]:
# Previsões com dados de teste
previsoes = modelo.transform(dados_teste)

In [38]:
previsoes

DataFrame[resultado: string, label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [39]:
previsoes.select("resultado", "label", "prediction", "probability").collect()

[Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9024, 0.0976, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9024, 0.0976, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9024, 0.0976, 0.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitori

In [40]:
# Avaliando a acurácia
avaliador = MulticlassClassificationEvaluator(predictionCol = "prediction", 
                                              labelCol = "label", 
                                              metricName = "accuracy")

In [41]:
avaliador.evaluate(previsoes)      

0.8823529411764706

In [42]:
# Resumindo as previsões - Confusion Matrix
previsoes.groupBy("label", "prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       2.0|    8|
|  0.0|       0.0|   11|
|  1.0|       1.0|   11|
|  0.0|       1.0|    2|
|  1.0|       0.0|    2|
+-----+----------+-----+



# Fim